In [27]:
# my version of the algorithm used in the baseline

from collections import defaultdict
import numpy as np
import scipy as sp
import cPickle as pickle
import time
import os
from __future__ import print_function

data_root = os.path.expanduser("~") + '/data/CSE255/'

In [28]:
start_time = time.time()
all_data = pickle.load(open(data_root + "all_data.pickle", "rb"))
print(time.time() - start_time)

15.4332668781


In [29]:
all_size = len(all_data)
train_size = 900000
valid_size = 100000
train_data = all_data[:train_size]
valid_data = all_data[all_size - valid_size:]

In [30]:
# get global average
all_helpfuls = np.array([d['helpful']['nHelpful'] for d in all_data])
all_outofs =  np.array([d['helpful']['outOf'] for d in all_data])
global_avg_ratio = np.sum(all_helpfuls) / np.sum(all_outofs.astype(float))
print('avg helpfulness ratio', global_avg_ratio)

avg helpfulness ratio 0.769983288753


In [31]:
# get average for a user
users_outof = dict()
users_helpful = dict()

for d in all_data:
    user_id = d['reviewerID']
    users_outof[user_id] = users_outof.get(user_id, 0.0) + float(d['helpful']['outOf'])
    users_helpful[user_id] = users_helpful.get(user_id, 0.0) + float(d['helpful']['nHelpful'])
    
users_ratio = dict()
for user_id in users_outof:
    if users_outof[user_id] != 0:
        users_ratio[user_id] = users_helpful[user_id] / users_outof[user_id]
    else:
        users_outof[user_id] = global_avg_ratio

In [33]:
# load 'pairs_Helpful.txt'
# get header_str and user_item_outofs
with open('pairs_Helpful.txt') as f:
    # read and strip lines
    lines = [l.strip() for l in f.readlines()]
    # stirip out the headers
    header_str = lines.pop(0)
    # get a list of user_item_ids
    user_item_outofs = [l.split('-') for l in lines]
    user_item_outofs = [[d[0], d[1], float(d[2])] for d in user_item_outofs]
    
# write to output file
f = open('predictions_Helpful.txt', 'w')
print(header_str, file=f)
for user_id, item_id, outof in user_item_outofs:
    if user_id in users_ratio:
        helpful_predict = users_ratio[user_id] * outof
    else:
        helpful_predict = global_avg_ratio * outof
    print('%s-%s-%s,%s' % (user_id, item_id, int(outof), helpful_predict), file=f)
f.close()